In [1]:
import cv2

import numpy as np
import pandas as pd
import pickle

from ultralytics import YOLO

from tqdm import tqdm

from utils import init_metadata

In [ ]:
metadata = init_metadata("./original-data/metadata_video.csv", "./original-data/metadata_imgs.csv")
paths = [path.replace("original-data", "processing-data") for path in metadata["metadata_video"]["path"].values]
path_labels = [path.replace("processing-data", "labels").replace(".mp4", ".pickle") for path in paths]

In [ ]:
with open("./labels/path_labels.pickle", "wb") as f:
    pickle.dump(path_labels, f)

In [3]:
path_labels[:5]

['labels/Южно-Сахалинск Панорама/video 19.2.66.10 - Cam-MBT-14 04.11.23 13.25.00-13.30.00.pickle',
 'labels/Южно-Сахалинск Панорама/video 19.2.66.9 - C-20-63 04.11.23 12.27.00-12.32.00.pickle',
 'labels/Южно-Сахалинск Панорама/video 19.2.66.9 - C-39-58 04.11.23 12.27.00-12.32.00.pickle',
 'labels/Южно-Сахалинск Панорама/video 19.2.66.9 - C-41-40 04.11.23 11.27.00-11.32.00.pickle',
 'labels/Южно-Сахалинск Панорама/video 19.2.66.10 - Cam-supermarket-123 04.11.23 10.29.00-10.34.00.pickle']

In [4]:
paths[:5]

['processing-data/Южно-Сахалинск Панорама/video 19.2.66.10 - Cam-MBT-14 04.11.23 13.25.00-13.30.00.mp4',
 'processing-data/Южно-Сахалинск Панорама/video 19.2.66.9 - C-20-63 04.11.23 12.27.00-12.32.00.mp4',
 'processing-data/Южно-Сахалинск Панорама/video 19.2.66.9 - C-39-58 04.11.23 12.27.00-12.32.00.mp4',
 'processing-data/Южно-Сахалинск Панорама/video 19.2.66.9 - C-41-40 04.11.23 11.27.00-11.32.00.mp4',
 'processing-data/Южно-Сахалинск Панорама/video 19.2.66.10 - Cam-supermarket-123 04.11.23 10.29.00-10.34.00.mp4']

In [5]:
model = YOLO("model/yolov8x.pt")

In [6]:
def marking_video(
    model: YOLO,
    path: str,
):
    results = []
    cap = cv2.VideoCapture(path)

    while(cap.isOpened()):
        ret, frame = cap.read()

        if ret:
            result = model.predict(frame, classes=0, device="0", verbose=False)

            results.append(
                {
                    "boxes": result[0].boxes,
                    "probs": result[0].probs
                }
            )

            
        else:
            break

    cap.release()
    
    return pd.DataFrame(results)


In [7]:
paths[0]

'processing-data/Южно-Сахалинск Панорама/video 19.2.66.10 - Cam-MBT-14 04.11.23 13.25.00-13.30.00.mp4'

In [8]:
for path, path_label in zip(tqdm(paths), path_labels):
    marks = marking_video(
        model=model,
        path=path
    )
    
    with open(path_label, "wb") as f:
        pickle.dump(marks, f)


 32%|███▏      | 239/737 [16:16:27<20:05:49, 145.28s/it][NULL @ 0x11af5e080] missing picture in access unit with size 1306
[h264 @ 0xa534880] no frame!
100%|██████████| 737/737 [47:01:37<00:00, 229.71s/it]   
